<a href="https://colab.research.google.com/github/myoungse/DeepLearning/blob/main/%ED%8C%80%ED%94%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [4]:
!pip install openai scikit-learn

In [3]:
from openai import OpenAI
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
from openai import OpenAI
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_qa_data(filenames):
    qa_data = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
            qa_data.extend(json.load(f))
    return qa_data

filenames = ['/content/sample_data/qa_dataset.json', '/content/sample_data/qa_dataset_p.json']
qa_data = load_qa_data(filenames)

questions = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(questions)

def find_best_answer(user_question):
    user_tfidf = vectorizer.transform([user_question])
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    best_match_index = cosine_similarities.argmax()
    return answers[best_match_index]

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    # 사용자 입력
    user_input = input("> ")

    # 가장 적합한 답변 찾기
    best_answer = find_best_answer(user_input)

    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": best_answer})

    # 응답 출력
    print(best_answer)

> 교내주차장 몇 시까지 운영해?
교내주차장은 공휴일 및 대체휴일, 평일에 24시간동안 운영합니다.
> 축제 기간 알려줘
지천관(지천관 부스)과 효민야구장 내(미락정 부스)에서 각각 부스를 운영합니다.


KeyboardInterrupt: Interrupted by user

In [10]:
pip install transformers

In [11]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [16]:
import json
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

def load_qa_data(filenames):
    qa_data = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
            qa_data.extend(json.load(f))
    return qa_data

filenames = ['/content/sample_data/qa_dataset.json', '/content/sample_data/qa_dataset_p.json']
qa_data = load_qa_data(filenames)

questions = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

question_embeddings = [get_bert_embeddings(question) for question in questions]

def find_best_answer(user_question):
    user_embedding = get_bert_embeddings(user_question)
    similarities = [cosine_similarity(user_embedding, q_emb).flatten()[0] for q_emb in question_embeddings]
    best_match_index = similarities.index(max(similarities))
    return answers[best_match_index]

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    # 사용자 입력
    user_input = input("> ")

    # 가장 적합한 답변 찾기
    best_answer = find_best_answer(user_input)

    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": best_answer})

    # 응답 출력
    print(best_answer)

KeyboardInterrupt: Interrupted by user

In [ ]:
import json
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI

def load_qa_data(filenames):
    qa_data = []
    for filename in filenames:
        with open(filename, 'r', encoding='utf-8') as f:
          qa_data.extend(json.load(f))
    return qa_data

filenames = ['/content/sample_data/qa_dataset.json', '/content/sample_data/qa_dataset_p.json', '/content/sample_data/qa_dataset_mili.json']
qa_data = load_qa_data(filenames)

questions = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

question_embeddings = [get_bert_embeddings(question) for question in questions]

def find_best_answer(user_question):
    user_embedding = get_bert_embeddings(user_question)
    similarities = [cosine_similarity(user_embedding, q_emb).flatten()[0] for q_emb in question_embeddings]
    best_match_index = similarities.index(max(similarities))
    return answers[best_match_index]

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

while True:
    # 사용자 입력
    user_input = input("> ")

    # 가장 적합한 답변 찾기
    best_answer = find_best_answer(user_input)

    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": best_answer})

    # 응답 출력
    print(best_answer)

> 예비군 몇 시까지 해?
당일기준 8시간 훈련이 이루어집니다.
